# Homework 4: Heroes of Pymoli

### Imports and Data

In [19]:
import pandas as pd 
import numpy as np
import json
data = pd.read_json('purchase_data.json')

### Player Count

In [2]:
# Player Count
def unique_players():
    labels = "Total Players"
    values = len(data['SN'].value_counts())
    
    df = pd.DataFrame({labels: values}, index=[0])
    return(df)

unique_players()

,Total Players
0,573


### Purchasing Analysis

In [3]:
# Total Purchases
def purchasing_totals():
    values = [len(data['Item Name'].value_counts()),
                        np.mean(data['Price']),
                        data.shape[0],
                        np.sum(data['Price'])]
    labels = ["Number of Unique Items", "Total Purchases", "Average Price", "Total Revenue"]
    
    df = pd.DataFrame({'Number of Unique Items': len(data['Item Name'].value_counts()),
                       'Average Price':np.mean(data['Price']),
                       'Total Purchases': data.shape[0],
                       'Total Revenue': np.sum(data['Price'])}, index=[0])
    df = df[labels]
    df.iloc[:,2:] = df.iloc[:,2:].applymap('${:,.2f}'.format)
    return(df)

purchasing_totals()

,Number of Unique Items,Total Purchases,Average Price,Total Revenue
0,179,780,$2.93,"$2,286.33"


### Gender Analysis

In [4]:
# Gender Demographics
def gender_demo():

    m = data.loc[data['Gender'] == 'Male']
    f = data.loc[data['Gender'] == 'Female']
    o = data.loc[data['Gender'] == 'Other / Non-Disclosed']
    mfo = [m, f, o]
    
    demo = [len(i['SN'].value_counts()) for i in mfo]
    total = np.sum(demo)
    percent = [round(demo[i]/total, 2) for i in [0, 1, 2]]
    
    df = pd.DataFrame({'Count': demo, 'Percent': percent})
    df.index = ['Male', 'Female', 'Other/Non-Disclosed']
    return(df)
    
gender_demo()

,Count,Percent
Male,465,0.81
Female,100,0.17
Other/Non-Disclosed,8,0.01


In [5]:
# Purchasing Analysis (Gender)
def purchasing_gender():
    col = ['Purchase Count', 'Purchasing Total', 'Purchase Average', 'Normalized Total']
    m = data.loc[data['Gender'] == 'Male']
    f = data.loc[data['Gender'] == 'Female']
    o = data.loc[data['Gender'] == 'Other / Non-Disclosed']
    mp = m['Price']
    fp = f['Price']
    op = o['Price']

    mfo = [m, f, o]
    mfop = [mp, fp, op]

    count = data['Gender'].value_counts()
    ave = [np.mean(i) for i in mfop]
    total = [np.sum(i) for i in mfop]
    
    unique = gender_demo().loc[:,'Count']
    norm = [total[i]/unique[i] for i in range(len(mfop))]

    df = pd.DataFrame({'Purchase Count': count, 'Purchasing Total': total, 'Purchase Average':ave, 
                       'Normalized Total': norm})
    df = df[col]
    df.iloc[:,2:] = df.iloc[:,2:].applymap('${:,.2f}'.format)
    df.index = ['Male', 'Female', 'Other/ Non-Disclosed']
    return(df)

purchasing_gender()

,Purchase Count,Purchasing Total,Purchase Average,Normalized Total
Male,633,1867.68,$2.95,$4.02
Female,136,382.91,$2.82,$3.83
Other/ Non-Disclosed,11,35.74,$3.25,$4.47


### Age Analysis

In [6]:
def age():
    labels = ['Purchase Count', 'Average Price', 'Total Purchase Value', 'Normalized Total Price']
    bins = [0, 10, 15, 20, 25, 30, 35, 40, 45]
    bin_names = range(8)
    data['Age Cat'] = pd.cut(data['Age'], bins, labels = bin_names)
    
    purchases = list(data['Age Cat'].value_counts().sort_index())
    ave_price = list(data.groupby('Age Cat')['Price'].mean())
    total_price = list(data.groupby('Age Cat')['Price'].sum())
    
    unique = list(data.groupby('Age Cat')['SN'].nunique())
    norm_tp = [total_price[i]/ unique[i] for i in range(8)]
    
    df = pd.DataFrame()
    df['Purchase Count'] = purchases
    df['Average Price']=ave_price
    df['Total Purchase Value']=total_price
    df['Normalized Total Price'] = norm_tp
    
    df.iloc[:,1:4] = df.iloc[:, 1:4].applymap('${:.2f}'.format)

    df.index = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '34-39', '40+']
    return(df)

age()

,Purchase Count,Average Price,Total Purchase Value,Normalized Total Price
<10,32,$3.02,$96.62,$4.39
10-14,78,$2.87,$224.15,$4.15
15-19,184,$2.87,$528.74,$3.80
20-24,305,$2.96,$902.61,$3.86
25-29,76,$2.89,$219.82,$4.23
30-34,58,$3.07,$178.26,$4.05
34-39,44,$2.90,$127.49,$5.10
40+,3,$2.88,$8.64,$2.88


In [7]:
list(data.groupby('Age Cat')['SN'].nunique())

[22, 54, 139, 234, 52, 44, 25, 3]

### Top Spending

In [8]:
def top_spending():    
    col = ['SN', 'Purchase Count', 'Average Purchase Price', 'Total Purchase Price']
    df = pd.DataFrame(np.sum(data.groupby('SN')['Price'])).reset_index()
    df = df.sort_values('Price', ascending=False)

    SN = list(df.head()['SN'])
    counts = [data[data['SN'] == SN[i]].shape[0] for i in range(len(SN))]
    total = list(df.head().loc[:,'Price'])

    counts = np.array(counts)
    total = np.array(total)
    average = total/counts


    df = pd.DataFrame({'SN': SN, 'Purchase Count': counts, 'Total Purchase Value': total,
                       'Average Purchase Price': average})
    df = df[['SN', 'Purchase Count', 'Total Purchase Value', 'Average Purchase Price']]
    
    df.iloc[:,2:] = df.iloc[:, 2:].applymap('${:.2f}'.format)
    return(df)

top_spending()

,SN,Purchase Count,Total Purchase Value,Average Purchase Price
0,Undirrala66,5,$17.06,$3.41
1,Saedue76,4,$13.56,$3.39
2,Mindimnya67,4,$12.74,$3.18
3,Haellysu29,3,$12.73,$4.24
4,Eoda93,3,$11.58,$3.86


### Popular Items

In [9]:
def pop_items():
    # Returns DataFrame containing column headers:
    col = ['Item ID', 'Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value']

    # Calculating values for df
    series = data['Item ID'].value_counts().head()
    items = list(series.index)

    temp = data[data['Item ID'].isin(items)]
    counts = np.array(temp['Item ID'].value_counts())
    names = list(temp['Item Name'].value_counts().index)

    total = [np.sum(temp[temp['Item ID'] == items[i]]['Price']) for i in range(len(items))]
    price = [temp[temp['Item ID'] == items[i]].reset_index().loc[0,'Price'] for i in range(len(items))]

    # Formatting DataFrame
    df = pd.DataFrame({'Item ID':items, 'Item Name': names, 'Purchase Count': counts, 'Item Price':price,
                 'Total Purchase Value': total})
    df = df[col]
    df.iloc[:,3:] = df.iloc[:, 3:].applymap('${:.2f}'.format)
    
    return(df)

pop_items()

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,84,Arcane Gem,11,$2.23,$24.53
1,39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
2,31,Trickster,9,$2.07,$18.63
3,34,Retribution Axe,9,$4.14,$37.26
4,175,Woeful Adamantite Claymore,9,$1.24,$11.16


### Profitable Items

In [10]:
def prof_items():
    col = ['Item ID', 'Item Name', 'Purchase Count', 'Item Price', 'Total Purchase Value']

    df = pd.DataFrame(np.sum(data.groupby('Item ID')['Price'])).reset_index()
    df = df.sort_values('Price', ascending = False)

    items = list(df.head()['Item ID'])
    counts = [data[data['Item ID'] == items[i]].shape[0] for i in range(len(items))]

    temp = data[data['Item ID'].isin(items)]
    names = list(temp['Item Name'].value_counts().index)

    total = list(df.head().loc[:,'Price'])
    price = [temp[temp['Item ID'] == items[i]].reset_index().loc[0, 'Price'] for i in range(len(items))]

    df = pd.DataFrame({'Item ID': items, 'Item Name': names, 'Purchase Count':counts, 'Item Price':price,
                     'Total Purchase Value':total})
    df = df[col]
    df.iloc[:,3:] = df.iloc[:,3:].applymap('${:.2f}'.format)
    return(df)

prof_items()

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,34,Retribution Axe,9,$4.14,$37.26
1,115,"Splitter, Foe Of Subtlety",7,$4.25,$29.75
2,32,Spectral Diamond Doomblade,6,$4.95,$29.70
3,103,Orenmir,6,$4.87,$29.22
4,107,Singed Scalpel,8,$3.61,$28.88


In [17]:
age().to_csv('Age.csv')
gender_demo().to_csv('Gender.csv')
purchasing_gender().to_csv('GenderPur.csv')